In [23]:
# get all the records from PG and do some random selection
import psycopg2 as pg
import pandas as pd
import pandas.io.sql as psql

# con = psycopg2.connect(database="celebration", user="celebrate", password="forever", host="192.168.1.162", port="5432")
# print("Database opened successfully")    

# cur = con.cursor()
# cur.execute(f"insert into song(song_title,song_added,song_class,song_offset_start,song_offset_end,song_notes) values ('{title}',now(),'testing123', {offset_start},{offset_end},'hi');")
# con.commit()
# print("Record inserted successfully")
# con.close()


connection = pg.connect("host=192.168.1.162 dbname=celebration user=celebrate password=forever")
dataframe = psql.read_sql('SELECT * FROM song', connection)
#product_category = psql.read_sql_query('select * from product_category', connection)

In [30]:
# pick only intro
df = dataframe
df1 = df[df.song_class == "intro"].sample(n=2)
df2 = df[df.song_class == "main"].sample(n=5)
df3 = df[df.song_class == "finish"].sample(n=2)

dff1 = pd.concat([df1,df2,df3])
dff2 = dff1.drop(columns=['song_added','song_notes'])

dff2

,song_id,song_title,song_class,song_offset_start,song_offset_end
24,40,Tangerine Dream - Ricochet (State Azure Cover)...,intro,147721,151380
22,37,Tenacious D Beelzeboss with lyrics-vOBKxUT9Da4...,intro,141484,143755
28,16,Ly-O-Lay Ale Loya (The Counterclockwise Circle...,main,29318,30609
31,29,Claire Hamill - Winter - Icicle Rain-_mH_TQgj8...,main,77143,77756
33,45,Our sacred earth www.tantra4jou.nl like this-j...,main,167070,189490
32,38,Mari Boine - Vuoi Vuoi Mu (Vuoi Vuoi Me)-NyB90...,main,143756,144928
30,24,Yeha Noha (Wishes Of Happiness And Prosperity)...,main,47045,47958
38,21,The Soul Force (Veeresh)-_8MMaFVzm3Y.mp4,finish,39569,43673
37,18,The Flaming Lips - How (Official Music Video)-...,finish,32247,34972
